In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/28 00:55:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
import pandas as pd

In [4]:
df = spark.read.parquet('fhvhv/2021/02/')

Question 1: PySpark Version

In [11]:
pyspark.__version__

'3.0.3'

Question 2: Size of fhvhv folder Feb 2021

In [9]:
!du -sh fhvhv/2021/02

210M	fhvhv/2021/02


Question 3: Record count on Feb 15, 2021

In [13]:
df.registerTempTable('fhvhv_data')

In [14]:
spark.sql("""
SELECT
    COUNT(*) as ct_feb15
FROM
    fhvhv_data
WHERE DATE(pickup_datetime) between '2021-02-15' and '2021-02-15'
""").show()

+--------+
|ct_feb15|
+--------+
|  367170|
+--------+



Question 4: Day with the longest trip

In [24]:
spark.sql("""
SELECT 
    pickup_datetime, ((bigint(to_timestamp(dropoff_datetime)))-(bigint(to_timestamp(pickup_datetime))))/(60) as duration
FROM
    fhvhv_data
ORDER BY 2 DESC
LIMIT 1
""").show()

+-------------------+--------+
|    pickup_datetime|duration|
+-------------------+--------+
|2021-02-11 13:40:44|  1259.0|
+-------------------+--------+



Question 5: stages (for most frequent `dispatching_base_num`)

In [26]:
spark.sql("""
SELECT 
    dispatching_base_num, 
    COUNT(*) as ct_dispatching_base_num
FROM 
    fhvhv_data
GROUP BY 1
""").show()

+--------------------+-----------------------+
|dispatching_base_num|ct_dispatching_base_num|
+--------------------+-----------------------+
|              B02876|                 215693|
|              B03136|                   1741|
|              B02877|                 198938|
|              B02869|                 429720|
|              B02883|                 251617|
|              B02835|                 189031|
|              B02884|                 244963|
|              B02880|                 115716|
|              B02878|                 305185|
|              B02836|                 128978|
|              B02872|                 882689|
|              B02512|                  41043|
|              B02867|                 200530|
|              B02866|                 311089|
|              B02871|                 312364|
|              B02889|                 138762|
|              B02844|                   3502|
|              B02510|                3233664|
|            

Question 6: Most common locations pair

In [5]:
df_zones = spark.read.parquet('zones/')

In [11]:
df.registerTempTable('df')
df_zones.registerTempTable('df_zones')

In [18]:
spark.sql("""
SELECT *
FROM df_zones
WHERE Zone LIKE "%East New York%"
""").show()

+----------+--------+--------------------+------------+
|LocationID| Borough|                Zone|service_zone|
+----------+--------+--------------------+------------+
|        76|Brooklyn|       East New York|   Boro Zone|
|        77|Brooklyn|East New York/Pen...|   Boro Zone|
+----------+--------+--------------------+------------+



In [26]:
spark.sql("""
SELECT concat_zones, COUNT(*) as ct_concat_zones
FROM
(SELECT dispatching_base_num, pickup_datetime, dropoff_datetime, PULocationID, PUzone, DOLocationID, Zone as DOzone,
    CONCAT(PUzone, ' / ', Zone) as concat_zones
FROM
(SELECT dispatching_base_num, pickup_datetime, dropoff_datetime, PULocationID, Zone as PUzone, DOLocationID
FROM
    df t
LEFT JOIN df_zones zpu ON t.PULocationID = zpu.LocationID) t2
LEFT JOIN df_zones zdo ON t2.DOLocationID = zdo.LocationID) t3
GROUP BY 1
ORDER BY 2 DESC
LIMIT 2
""").show()

+--------------------+---------------+
|        concat_zones|ct_concat_zones|
+--------------------+---------------+
|East New York / E...|          45041|
|Borough Park / Bo...|          37329|
+--------------------+---------------+




[Stage 23:====================================================> (193 + 4) / 200]



EXTRA - try join df's first

In [6]:
df_join1 = df.join(df_zones, df.PULocationID == df_zones.LocationID)

In [7]:
df_join2 = df_join1.join(df_zones, df_join1.DOLocationID == df_zones.LocationID)

In [8]:
df_join2.registerTempTable('df_join2')